# Regresión logística
La regresión logística (_Logistic Regression_) es un típo de regresión cuyo objetivo es determinar la probabilidad de que una instancia pertezca a una clase $y$, dado un conjunto de variables independientes $x_i$ que la definen. En este contexto, las instancias están representadas como un vector de variables independientes $x=\mathbb{R}^{n}$ y una clase $y=\{0,1\}$. Es decir:

$$P(y|x)=h(x)$$

En este contexto, la función seleccionada para hacer esta estimación por excelencia es la sigmoide.

$$sigmoid(z)=\frac{1}{1+e^{-z}}$$

## Ejercicio
Grafíque la función sigmoid con $z$ en el rango $[-6, 6]$.

In [ ]:
!pip install tqdm
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report


#Ingrese su código aquí
def sigmoid(x):
    return 0

x = np.linspace(-6, 6, 250)
plt.plot(x, sigmoid(x))
plt.show()

Además de que la imagen de esta función está en $(0, 1)$, la derivada de esta función es:
$$\frac{sigmoid(z)}{dz}=sigmoid(z)(1-sigmoid(z))$$
Lo que facilitaba su implementación.

En este contexto, $z$ es una convinación lineal de las variables $x$.


## Función de error
Para calcular el error, se utiliza la entropía cruzada entre el valor esperado y el valor obtenido.
$$CE(y,\hat{y})=\frac{\sum(-y*log(\hat{y})-(1-y)*log(1-\hat{y}))}{N}$$
En este contexto, la entropía curzada se interpreta como la información promedio (en bits) necesaria para determinar el valor de $y$ dado que se conoce el valor de $\hat{y}$.

__Nota__: Por simplicidad, se interpreta el logaritmo como logaritmo natural, pero se puede usar logaritmo en cualquier base, ya que solo afecta en una constante.

__Ejercicio Opcional__: Calcule  $\frac{d(CE(y,\hat{y}))}{d\hat{y}}$, $\frac{d(CE(y,\hat{y}))}{dw}$ $\frac{d(CE(y,\hat{y}))}{db}$. Asuma que $w$ es un escalar y considere que $y$ es $0$ o $1$. 

## Ejemplo
Para el ejemplo de regresión logística se utilizará el conjunto de datos de cancer de pecho provisto. Este conjuntos de datos fue recolectado por investigadores de la Universisda de Wisconsin y provisto por la [UCI](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)). Para acceder al conjunto de datos, no es necesario descargarlo y convertirlo al formato, ya que en encuentra provisto por el módulo de [_sklearn.datasets_](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer) de la librería sickit-learn, que es una librería de _machine learning_ que se utilizará durante el curso.

El dataset tiene 569 instancias, con 30 atributos cada una. Las instancias pueden ser clasificadas entre Malignos y Benignos. El dataset está ligeramente desbalanceado, lo que significa que existen más instancias de una clase que de la otra. En partícular, 37,25% de las instancias son Malignas y 62,75% son Benignas. La siguiente tabla resume el conjunto de datos:

| Propiedad | Valor |
| --- | --- |
| Clases | 2 |
| Ejemplos por clase | 212(M-0), 357(B-1) | 
| Total de instancias | 569 |
| Dimensionalidad | 30|

El siguiente código:
1. Levanta los datos divididos en `x` (atributos) e `y` (clase).
1. Divide los datos en entrenamiento y testing.
1. Escala los datos de entrenamiento a valores entre 0 y 1.
1. Aplica las correcciones de escalado al conjunto de testing.

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import confusion_matrix


x, y = load_breast_cancer(True)
x_train = x[:500,:]
y_train = y[:500]
x_test = x[500:,:]
y_test = y[500:]

maxs = np.max(x_train, axis=0)
mins = np.min(x_train, axis=0)
x_train = (x_train - mins) / (maxs - mins)
x_test = (x_test - mins) / (maxs - mins)

### Visualizando el problema
Muchas veces la visualización del problema ayuda a entender sus carácteristicas. Sin embargo, visualizar elementos en 30 dimensiones no es posible. Para esto existen diferentes técnicas que nos permiten reducir la dimensionalidad de los elementos manteniendo ciertas propiedades. Para visualizar los datos utilizaremos T-distributed Stochastic Neighbor Embedding (t-SNE) [1], que reduce la dimensiones manteniendo la distribución estadistica de las distancias entre los puntos de datos. 

El siguiente código aplica t-SNE a los datos de entrenamiento y los grafica utilizando estrellas azules para los benignos y rojas para los malignos. Es importante notar que los puntos obtenidos varian de ejecución en ejecución ya que t-SNE es sensible a su inicialización.

[1] van der Maaten, L.J.P.; Hinton, G.E. (Nov 2008). ["Visualizing High-Dimensional Data Using t-SNE"](http://jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf). Journal of Machine Learning Research. 9: 2579–2605.

In [ ]:
from sklearn.manifold import TSNE

ts_rep = TSNE().fit_transform(x_train)
for point, label in zip(ts_rep, y_train):
    rep = 'b*' if label == 1 else 'r*'
    plt.plot([point[0]], [point[1]], rep)
plt.show()

Una vez visualizados los datos, hacemos una regresión logística sobre los datos. Para analizar el funcionamiento de la regresíon logística utilizaremos una matrix de confusión. En esta matriz las filas representan las clases reales y las columnas las clases predichas. En cada celda se encuentran la cantidad de instancias con la clase de la fila y la predicción asociada de la columna.

| | Predición: 0  | Predición: 1 |
| --- | --- | --- |
| __Real: 0__ | Verdaderos Negativos | Falsos Positivos |
| __Real: 1__ | Falsos Negativos | Verdaderos Positivos |

Esta matriz permite visualizar fácilmente los valores de:
* __Verdaderos Negativos__: cantidad de instancias clasificadas como negativas que efectivamente eran negativas. En nuestro ejemplo Maligno es la clase negativa porque tiene asociada el cero. Generalmente referido como TN (_True Negative_).
* __Verdaderos Positivos__: cantidad de instancias clasificadas como positivas que efectivamente eran positivas. En nuestro ejemplo Benigno es la clase poasitiva porque tiene asociada el uno. Generalmente referido como TP (_True Positive).
* __Falsos Negativos__: cantidad de instancias clasificadas como negativas, pero en la realidad era positivas. También llamado Error de Tipo II en estadística. Generalmente referido como FN (_False Negative_).
* __Falsos Positivos__: cantidad de instancias clasificadas como positivas, pero en la realidad era negativos. También llamado Error de Tipo I en estadística. Generalmente referido como FP (_False Positive_).


__Nota__: Como la regresión logística retorna un valor probabilistico se seleccionó un umbral de 0.5 para discernir entre clasificaciones positivas y negativas.

In [ ]:
import tensorflow as tf

'''Esta función dibuja bonita la matríz de confunsión.
'''
def show_confusion_matrix(cm, labels):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(cm)
    plt.title('Matriz de confusión')
    fig.colorbar(cax)
    ax.set_xticklabels([''] + labels)
    ax.set_yticklabels([''] + labels)
    plt.xlabel('Predicho')
    plt.ylabel('Verdadero')
    for i, row in zip(range(len(cm)), cm):
        for j, val in zip(range(len(row)), row):
            ax.text(i, j, str(val), va='center', ha='center').set_backgroundcolor('white')
    plt.show()

In [ ]:
def logistic_regression(x, w, b):
    return 1/(1+tf.exp(-(tf.matmul(x, w) + b)))[:,0]

def crossentropy(yt, yp):
    return tf.math.reduce_mean(-yt*tf.math.log(tf.clip_by_value(yp, 1e-6, 1)) - (1-yt)*tf.math.log(tf.clip_by_value(1-yp, 1e-6, 1)))

In [ ]:
x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_test = y_test.astype(np.float32)

In [ ]:
w = tf.random.uniform(shape=[30, 1], minval=-1, maxval=1)
b = tf.random.uniform(shape=[], minval=-1, maxval=1)

y_pred = logistic_regression(x_test, w, b)
show_confusion_matrix(confusion_matrix(y_test, y_pred > 0.5), labels=['Maligno', 'Benigno'])

ciclos = 1000
lr = 0.1 
errors = []
for i in tqdm(range(ciclos)):
    with tf.GradientTape() as g:
        g.watch([w, b])
        loss = crossentropy(y_train, logistic_regression(x_train, w, b))
        errors.append(loss.numpy())
    gw, gb = g.gradient(loss, [w, b])
    w = w - lr * gw
    b = b - lr * gb

print('Errores a medida que se actualiza el valor de w')
plt.plot(errors)
plt.show()
print('El w final es {}'.format(w))
print('El b final es {}'.format(b))

y_pred = logistic_regression(x_test, w, b).numpy()
show_confusion_matrix(confusion_matrix(y_test, y_pred > 0.5), labels=['Maligno', 'Benigno'])

## Stochastic Gradient Descent
Hasta el momento calculamos el gradiente sobre todo el conjunto de datos de entrenamiento. Sin embargo, esto puede no ser posible o eficiente cuando tenemos un conjunto de datos grande. Por esto, se suele dividir el conjunto de datos en mini-batch y entrenar sobre estos mini-batchs. La idea, es que en promedio la agregación de los efectos de la actualización en cada mini-batch nos acerque al mínimo. Obviamente, esto no significa que cada actualización nos acerque al mínimo global.


In [ ]:
from sklearn.utils import shuffle

w = tf.random.uniform(shape=[30, 1], minval=-1, maxval=1)
b = tf.random.uniform(shape=[], minval=-1, maxval=1)
ciclos = 100
lr = 0.1 
errors = []
errors_minibatch = []
errors_minibatch2 = []
for i in range(ciclos):
    x_s, y_s = shuffle(x_train, y_train)
    for mini_batch in range(0, 500, 50):
        with tf.GradientTape() as g:
            g.watch([w, b])
            loss = crossentropy(y_s[mini_batch:mini_batch+50], logistic_regression(x_s[mini_batch:mini_batch+50], w, b))
            errors_minibatch2.append(loss.numpy())
            errors_minibatch.append(crossentropy(y_train, logistic_regression(x_train, w, b)).numpy())
        gw, gb = g.gradient(loss, [w, b])
        w = w - lr * gw
        b = b - lr * gb
    errors.append(crossentropy(y_train, logistic_regression(x_train, w, b)).numpy())

print('Errores a medida que se actualiza el valor de w')
plt.plot(errors)
plt.show()
plt.plot(errors_minibatch)
plt.show()
plt.plot(errors_minibatch2)
plt.show()
print('El w final es {}'.format(w))
print('El b final es {}'.format(b))
y_pred = logistic_regression(x_test, w, b).numpy()
show_confusion_matrix(confusion_matrix(y_test, y_pred > 0.5), labels=['Maligno', 'Benigno'])

## Métricas
La matriz de confusión es buena para visualizar las capacidades de un clasificador, pero dificulta comparar sus capacidades contra otros modelos. Por este motivo, se han desarrollado diversas métricas para evaluar cuan bien funciona un clasificador. Algunas de esta son:
* Accuracy: Porcentaje de predicciones correctas $acc=\frac{TP+TN}{TP+TN+FP+FN}$
* Precision: Porcentaje de predicciones positivas correctas $precision=\frac{TP}{TP+FP}$
* Recall: Porcentaje de las instancias pertenecientas a la clase positiva que fueron clasificadas correctamentes $recall=\frac{TP}{TP+FN}$
* F1-measure: Media harmónica entre la Presicion y el Recall. $F1=2\frac{precision \times recall}{precision+recall}$
* Matthews correlation coefficient: Es la correlación entre las predicciones y los valores reales. 1 indica predicción perfecta, -1 indica total desacuerdo entre lo predicho y lo real, y 0 indica que el clasificador no aporta ninguna información $MCC = \frac{ TP \times TN - FP \times FN } {\sqrt{ (TP + FP) ( TP + FN ) ( TN + FP ) ( TN + FN ) } }$

### Ejercicio
Implemente las métricas y apliquelas sobre los resultados del clasificador de ejemplo.
__Recuerde__: en el futuro podrá utilizar las implementaciones provistas por scikit-learn.

In [ ]:
def tp_tn_fn_fp(y_true, y_pred):
    y_pred = y_pred > 0.5
    tp = np.sum(y_true * y_pred)
    tn = np.sum((1-y_true) * (1-y_pred))
    fp = np.sum((1-y_true) * y_pred)
    fn = np.sum(y_true * (1-y_pred))
    return tp, tn, fn, fp

print(tp_tn_fn_fp(y_test, y_pred))

#Defina las metricas

## Problema de OCR de dígitos
Para este trabajos utilizaremos el conjunto de datos conocido como [MNIST](http://yann.lecun.com/exdb/mnist/). Este conjunto de datos ya se encuentra dividido entre entrenamiento y testing. El problema consiste en clasificar imagenes de dígistos escritos a mano al dígito correspondiente.

| Propiedad | Valor |
| --- | --- |
| Clases | 10 |
| Tamaño de las imagenes | 28 X 28 |
| Instancias de entrenemiento | 60.000 |
| Instancias de validación | 10.000 |
| Valor mínimo de cada pixel | 0 |
| Valor máximo de cada pixel | 255 |

A continuación se carga el dataset y se dibujan los primeros 100 ejemplos del conjunto de entrenamiento.

## Clasificación multiclase
Para la clasificación multiclase una función muy utilizada es el softmax. Esta función toma por entrada un vector y retorna un vectos tal que la suma de todos sus elementos es $1$ y todos los valores están entre $0$ y $1$. Si nuestro problema de clasificación tiene n clases, podemos usar la función softmax y un vector n-dimensiones. Entonces, podemos interpretar la salida de esta función como la distribución de probabilidades de las clases.
$$softmax_i(x) = \frac{e^{x}}{\sum e^{x}} $$

## Categorical Crossentropy
Esta función de error considera el error sobre la categoría real, normalizando el valor de la predicción. Considerando $\hat{y}=(\hat{y}_1, \hat{y}_2, ..., \hat{y}_C)$ in vector de valores asociados a las clases
$$P_\hat{y}=\frac{\hat{y}}{\sum\hat{y}_i}$$
$$CCE(y,P_\hat{y})=-\frac{\sum y * log(P_\hat{y})}{N} $$
Notese que el valor de error se considera solo sobre las clases verdaderas, las otras son afectadas a través de la normalización de la salida $\hat{y}$.




In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('100 primeros elementos del conjunto de entrenaimento')
f = plt.figure(111)
for i in range(10):
    for j in range(10):
        ax = f.add_subplot(10, 10, i + j*10 + 1)
        ax.set_xticklabels('')
        ax.set_yticklabels('')
        ax.imshow(x_train[i + j*10, :, :], cmap='gray')
plt.show()
print(y_train[:100])

In [ ]:
size = x_train.shape[1]*x_train.shape[2]
x_train = x_train.reshape((x_train.shape[0], size)) / 255
x_test = x_test.reshape((x_test.shape[0], size)) / 255

yc_train, yc_test = to_categorical(y_train), to_categorical(y_test)

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
yc_train = yc_train.astype(np.float32)
yc_test = yc_test.astype(np.float32)

In [ ]:
def softmax(z):
    exp = tf.exp(z)
    return exp / tf.expand_dims(tf.math.reduce_sum(exp, axis=1), axis=-1)

def predict(x, w, b):
    return softmax(tf.matmul(x, w) + b)

def categorical_crossentropy(y_true, y_pred):
    return tf.math.reduce_mean(-tf.math.reduce_sum(y_true * tf.math.log(tf.clip_by_value(y_pred, 1e-6, 1)), axis=1))

def loss_f(y_true, x, w, b):
    return categorical_crossentropy(y_true, predict(x, w, b))

In [ ]:
w = tf.random.uniform(shape=[size, 10], minval=-1, maxval=1)
b = tf.random.uniform(shape=[10], minval=-1, maxval=1)
ciclos = 100
lr = 0.01 
batch_size = 500
errors = []
errors_minibatch = []
vw = tf.zeros(shape=[size, 10])
vb = tf.zeros(shape=[10])
for i in tqdm(range(ciclos)):
    x_s, y_s = shuffle(x_train, yc_train)
    for mini_batch in range(0, x_s.shape[0], batch_size):
        with tf.GradientTape() as g:
            g.watch([w, b])
            loss = loss_f(y_s[mini_batch:mini_batch+batch_size], x_s[mini_batch:mini_batch+batch_size], w, b)
            errors_minibatch.append(loss.numpy())
        gw, gb = g.gradient(loss, [w, b])
        w = w - lr * gw
        b = b - lr * gb
    errors.append(loss_f(yc_train, x_train, w, b).numpy())
    
print('Errores a medida que se actualiza el valor de w')
plt.plot(errors)
plt.show()
plt.plot(errors_minibatch)
plt.show()
print('El w final es {}'.format(w))
print('El b final es {}'.format(b))
y_pred = predict(x_test, w, b)
show_confusion_matrix(confusion_matrix(y_test, np.argmax(y_pred , axis=1)), labels=[str(i) for i in range(10)])
print(classification_report(y_test, np.argmax(y_pred , axis=1)))

## Momentum
Para mejorar los resultados una técnica muy utilizada es agregar un momentum. La idea del momentum es que vaya llevando una historia del movimiento entre los mini-batchs, y como en promedio esperamos que nos lleve a un mínimo global el momentum llevaría nuestras actualizaciones más hacia el este mínimo.

$$vel_n=momentum * vel_{n-1} – lr * grad_n$$

$$w_{n+1} = w_{n} + vel_n$$ 

In [ ]:
w = tf.random.uniform(shape=[size, 10], minval=-1, maxval=1)
b = tf.random.uniform(shape=[10], minval=-1, maxval=1)
ciclos = 100
lr = 0.01 
batch_size = 500
momentum = 0.9
errors = []
errors_minibatch = []
vw = tf.zeros(shape=[size, 10])
vb = tf.zeros(shape=[10])
for i in tqdm(range(ciclos)):
    x_s, y_s = shuffle(x_train, yc_train)
    for mini_batch in range(0, x_s.shape[0], batch_size):
        with tf.GradientTape() as g:
            g.watch([w, b])
            loss = loss_f(y_s[mini_batch:mini_batch+batch_size], x_s[mini_batch:mini_batch+batch_size], w, b)
            errors_minibatch.append(loss.numpy())
        gw, gb = g.gradient(loss, [w, b])
        vw = momentum * vw - lr * gw
        vb = momentum * vb - lr * gb
        w = w + vw
        b = b + vb
    errors.append(loss_f(yc_train, x_train, w, b).numpy())
    
print('Errores a medida que se actualiza el valor de w')
plt.plot(errors)
plt.show()
plt.plot(errors_minibatch)
plt.show()
print('El w final es {}'.format(w))
print('El b final es {}'.format(b))
y_pred = predict(x_test, w, b)
show_confusion_matrix(confusion_matrix(y_test, np.argmax(y_pred , axis=1)), labels=[str(i) for i in range(10)])
print(classification_report(y_test, np.argmax(y_pred , axis=1)))